<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_agentchat_groupchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat

Modified based on https://github.com/microsoft/FLAML/blob/4ea686af5c3e8ff24d9076a7a626c8b28ab5b1d7/notebook/autogen_multiagent_roleplay_chat.ipynb

## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [autogen] option:
```bash
pip install flaml[autogen]
```

In [1]:
%%capture --no-stderr
# %pip install flaml[autogen]~=2.0.0rc4

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
from flaml import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt4", "gpt-4-32k", "gpt-4-32k-0314"],
    },
)
# config_list_gpt35 = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": {
#             "gpt-3.5-turbo",
#             "gpt-3.5-turbo-16k",
#             "gpt-3.5-turbo-0301",
#             "chatgpt-35-turbo-0301",
#             "gpt-35-turbo-v0301",
#         },
#     },
# )

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct Agents

In [3]:
import sys

llm_config = {"config_list": config_list_gpt4}
room = autogen.Room()
chat_manager = autogen.ChatManagerAgent(room=room, max_round=3)
human = autogen.GroupChatParticipant(
   name="Human",
   system_message="A human admin.",
   human_input_mode="ALWAYS",
   llm_config=False,
   chat_manager=chat_manager,
)
alice = autogen.GroupChatParticipant(
    name="Alice",
    system_message=autogen.AssistantAgent.DEFAULT_SYSTEM_MESSAGE,
    max_consecutive_auto_reply=sys.maxsize,
    human_input_mode="NEVER",
    llm_config=llm_config,
    code_execution_config=False,
    chat_manager=chat_manager,
)
bob = autogen.GroupChatParticipant(
    name="Bob",
    system_message="Code reviewer. Prevent code execution if unsafe or not well documented. Suggest changes. Otherwise, approve and return the final code to execute.",
    max_consecutive_auto_reply=sys.maxsize,
    human_input_mode="NEVER",
    llm_config=llm_config,
    code_execution_config=False,
    chat_manager=chat_manager,
)

room.agents = [human, alice, bob]

## Start Chat

In [4]:
room.send("find a latest paper about generative agents", chat_manager)

room (to chat_manager):

find a latest paper about generative agents

--------------------------------------------------------------------------------
chat_manager (to room):

find a latest paper about generative agents

--------------------------------------------------------------------------------
room (to Human):

find a latest paper about generative agents

--------------------------------------------------------------------------------
room (to Alice):

find a latest paper about generative agents

--------------------------------------------------------------------------------
room (to Bob):

find a latest paper about generative agents

--------------------------------------------------------------------------------
chat_manager (to Human):

You are in a role play game. Read the following conversation.
Then select the next role from ['Human', 'Alice', 'Bob'] to play. Only return the role.

--------------------------------------------------------------------------------


Human (to chat_manager):

Alice

--------------------------------------------------------------------------------
chat_manager (to Alice):

speak

--------------------------------------------------------------------------------
[{'content': 'You are a helpful AI assistant.\n    In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute. You must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can\'t modify your code. So do not suggest incomplete code which requires users to modify. Don\'t use a code block if it\'s not intended to be executed by the user.\n    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time.\n    2. When you need to perform some 